In [162]:
import pandas as pd
import numpy as np
import json

# from scipy.cluster.vq import kmeans

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

from tqdm import tqdm

import seaborn as sns

words = {'decided': 881, 'give': 1399, 'house': 1603, 'another': 171, 'shot': 2833, 'live': 1858, 'neighborhood': 2090,
         'took': 3281, 'our': 2201, 'friends': 1346, 'young': 3622, 'daughter': 866, 'slow': 2893, 'never': 2094,
         'drinks': 1024, 'forever': 1311, 'come': 687, 'asked': 233, 'check': 585, 'said': 2690, 'down': 1003,
         'delay': 895, 'sure': 3134, 'why': 3542, 'couldn': 782, 'offered': 2153, 'information': 1669, 'earlier': 1048,
         'top': 3283, 'off': 2151, 'entire': 1113, 'party': 2262, 'sick': 2846, 'after': 110, 'dinner': 956, 'ate': 245,
         'same': 2700, 'thing': 3228, 'needless': 2086, 'say': 2728, 'll': 1863, 'quick': 2498, 'face': 1188,
         'nachos': 2066, 'dining': 955, 'wait': 3453, 'see': 2765, 'how': 1604, 'rate': 2514, 'spot': 2995,
         'know': 1767, 'says': 2730, 'once': 2172, 'twice': 3344, 'before': 337, 'beer': 333, 'events': 1135,
         'truly': 3329, 'impressed': 1641, 'tend': 3196, 'super': 3125, 'hype': 1620, 'something': 2932, 'sour': 2955,
         'tap': 3173, 'then': 3222, 'arrive': 223, 'four': 1324, 'people': 2292, 'enough': 1106, 'staff': 3008,
         'room': 2657, 'list': 1851, 'normally': 2119, 'stick': 3053, 'around': 220, 'single': 2865, 'call': 503,
         'night': 2103, 'happened': 1494, 'visit': 3443, 'figured': 1248, 'hey': 1547, 'eats': 1065, 'three': 3242,
         'us': 3404, 'head': 1511, 'over': 2210, 'two': 3346, 'girlfriends': 1397, 'drink': 1021, 'nothing': 2128,
         'fancy': 1205, 'enjoyed': 1102, 'split': 2988, 'chili': 611, 'sweet': 3145, 'potato': 2400, 'pretty': 2429,
         'fast': 1212, 'front': 1349, 'though': 3238, 'music': 2061, 'loud': 1886, 'could': 781, 'hardly': 1502,
         'hear': 1518, 'each': 1045, 'talk': 3168, 'surprising': 3138, 'tuesday': 3334, '8pm': 44, 'wasn': 3485,
         'terribly': 3203, 'crowded': 827, 'made': 1912, 'volume': 3449, 'situation': 2872, 'worse': 3585,
         'arrived': 224, 'loaded': 1866, 'initially': 1676, 'exciting': 1156, 'alas': 129, 'pushed': 2487,
         'noticed': 2130, 'fairly': 1195, 'toppings': 3286, 'pet': 2308, 'hunt': 1616, 'philly': 2313, 'warm': 3476,
         'shame': 2810, 'server': 2792, 'kind': 1756, 'little': 1857, 'still': 3057, 'conversation': 764, 'he': 1510,
         'eventually': 1136, 'delicious': 899, 'meaty': 1965, 'hearty': 1522, 'serving': 2796, 'bad': 277,
         'either': 1079, 'used': 3406, 'sauce': 2717, 'texture': 3208, 'point': 2376, 'think': 3230, 'biggest': 362,
         'feeling': 1234, 'definitely': 892, 'ok': 2163, 'city': 638, 'full': 1359, 'amazing': 156, 'isn': 1705,
         'going': 1414, 'bring': 443, 'hurry': 1617, 'heard': 1519, 'fried': 1341, 'pickles': 2324, 'insane': 1678,
         'do': 986, 'make': 1919, 'first': 1268, 'choice': 621, 'update': 3393, 'happy': 1499, 'hour': 1601,
         'craft': 803, 'draft': 1011, 'minus': 2015, 'hand': 1482, '20': 14, 'beers': 334, 'half': 1477, '30': 25,
         'yes': 3616, 'early': 1049, 'side': 2847, 'awesome': 269, 'way': 3500, 'those': 3237, 'might': 1998,
         'otherwise': 2200, 'wine': 3552, 'experiences': 1170, 'both': 410, 'times': 3259, 'monday': 2038,
         'ended': 1093, 'long': 1875, 'take': 3163, 'okay': 2164, 'fries': 1347, 'table': 3156, 'portions': 2393,
         'weren': 3523, 'huge': 1609, 'didn': 940, 'big': 360, 'problem': 2447, 'awhile': 271, 'everyone': 1141,
         'started': 3023, 'eating': 1064, 'soup': 2953, 'day': 867, 'cheese': 592, 'mac': 1908, 'loved': 1891,
         'creamy': 814, 'pieces': 2332, 'thought': 3239, 'touch': 3293, 'meatballs': 1963, 'sandwich': 2707,
         'size': 2874, 'fish': 1269, 'tiny': 3261, 'previous': 2430, 'reviewer': 2625, 'mentioned': 1982,
         'salads': 2693, 'burger': 475, 'liked': 1839, 'went': 3522, 'again': 113, 'sunday': 3122, '12': 5,
         'brunch': 455, 'honestly': 1580, 'every': 1138, 'leave': 1812, 'being': 344, 'omelet': 2169, 'onion': 2174,
         'sausage': 2720, 'came': 509, 'apple': 204, '11': 4, '50': 34, 'regular': 2569, '10': 1, 'build': 467,
         'own': 2224, 'variety': 3418, 'tasty': 3183, 'plenty': 2371, 'chunks': 632, 'lamb': 1785, 'poached': 2374,
         'egg': 1074, 'potatoes': 2401, 'smaller': 2897, 'looked': 1878, 'salmon': 2695, 'cream': 813, 'charge': 572,
         'coffee': 672, 'several': 2806, 'cups': 841, 'bloody': 383, 'mary': 1942, 'blown': 386, 'away': 268,
         'solid': 2927, 'manager': 1928, 'everything': 1142, 'perfect': 2297, 'ventured': 3431, 'opening': 2180,
         'others': 2199, 'extremely': 1184, 'excited': 1155, 'new': 2096, 'looking': 1879, 'favorite': 1221,
         'ran': 2505, 'past': 2268, 'few': 1243, 'while': 3532, 'under': 3364, 'than': 3211, 'open': 2178,
         'probably': 2446, 'should': 2835, 'cool': 773, 'menu': 1984, 'nowhere': 2132, 'found': 1322, 'guys': 1472,
         'put': 2488, 'name': 2067, 'hostess': 1598, 'told': 3272, 'him': 1557, '45': 31, 'minute': 2016,
         'willing': 3547, 'bar': 295, 'extensive': 1180, 'cocktails': 669, 'snacks': 2913, 'forward': 1321,
         'meal': 1954, 'waited': 3454, 'almost': 144, 'hours': 1602, 'gone': 1417, 'since': 2863, 'frustrated': 1353,
         'mostly': 2045, 'disappointment': 969, 'hunger': 1614, 'overly': 2217, 'yet': 3618, 'approached': 209,
         'let': 1826, 'leaving': 1814, 'particular': 2257, 'basically': 306, 'tired': 3265, 'hearing': 1520,
         'complaints': 715, 'totally': 3292, 'planning': 2353, 'coming': 694, 'until': 3390, 'run': 2678, 'such': 3105,
         'quality': 2492, 'corner': 776, 'area': 214, 'love': 1890, 'atmosphere': 247, 'friendly': 1345, 'course': 790,
         'finally': 1255, 'finished': 1263, 'couple': 787, 'where': 3530, 'cocktail': 668, 'points': 2378,
         'salad': 2692, 'sample': 2701, 'brought': 453, 're': 2525, 'vegan': 3422, 'breakfast': 433, 'reasonable': 2537,
         'many': 1933, 'plates': 2358, 'compared': 706, 'nearby': 2079, 'tea': 3185, 'places': 2348, 'limited': 1846,
         'vegetarian': 3426, 'options': 2190, 'trying': 3332, 'further': 1364, 'enjoying': 1103, 'friend': 1342,
         'last': 1791, 'week': 3509, 'during': 1042, 'requested': 2601, 'outside': 2207, 'seating': 2758, 'host': 1597,
         'special': 2968, 'brew': 438, 'water': 3493, 'actually': 80, 'received': 2543, 'much': 2054, 'flavor': 1280,
         'tasted': 3177, 'cucumbers': 836, 'taste': 3175, 'pickle': 2322, 'based': 304, 'experience': 1168,
         'stars': 3021, 'want': 3472, 'given': 1400, 'shots': 2834, 'unfortunately': 3376, 'street': 3079, 'most': 2044,
         'recent': 2545, 'specifically': 2972, 'old': 2165, 'school': 2737, 'game': 1369, 'group': 1459,
         'phillies': 2312, 'bartender': 301, 'hook': 1584, 'system': 3154, 'ten': 3195, 'minutes': 2017, 'post': 2397,
         'busy': 487, 'policy': 2380, 'happens': 1496, 'fine': 1258, 'rather': 2516, 'decide': 880, 'myself': 2065,
         'whether': 3531, 'patrons': 2280, 'plus': 2372, 'sitting': 2871, 'high': 1550, 'tables': 3157, 'large': 1789,
         'available': 261, 'handling': 1487, 'rest': 2613, 'restaurant': 2614, 'mood': 2042, 'gin': 1393,
         'infused': 1672, 'cucumber': 835, 'bit': 366, 'st': 3007, 'flat': 1279, 'she': 2818, 'similar': 2859,
         'club': 661, 'tasting': 3182, 'despite': 931, 'door': 997, 'hot': 1599, 'inside': 1680, 'air': 125, 'ac': 55,
         'uncomfortable': 3363, 'second': 2760, 'did': 939, 'enjoy': 1100, 'evening': 1133, 'servers': 2793,
         'wouldn': 3589, 'opposed': 2185, 'grabbing': 1430, 'bite': 367, 'won': 3566, 'expecting': 1166,
         'experienced': 1169, 'benefit': 349, 'doubt': 1000, 'tad': 3160, 'south': 2956, 'spring': 2998, 'things': 3229,
         'may': 1951, 'different': 946, 'any': 176, 'maybe': 1952, 'seated': 2757, 'quickly': 2499, 'downhill': 1004,
         'absolutely': 53, 'lunch': 1905, 'offer': 2152, 'items': 1710, 'interested': 1691, 'brews': 439, 'decent': 878,
         'selection': 2776, 'killed': 1754, 'roll': 2651, 'eyes': 1186, 'waiter': 3455, 'wanted': 3473, 'right': 2639,
         'hungry': 1615, 'exactly': 1146, 'obviously': 2142, 'spend': 2976, 'money': 2039, 'deal': 872, 'upset': 3400,
         'turn': 3337, 'fun': 1361, 'sat': 2710, 'slowly': 2895, 'bringing': 444, 'idea': 1625, 'next': 2099,
         'guy': 1471, 'orders': 2193, 'find': 1256, 'ready': 2530, 'bill': 364, 'including': 1653, 'waiting': 3457,
         'roast': 2647, 'pork': 2390, 'rolls': 2654, 'toasted': 3267, 'mussels': 2062, 'bland': 375, 'veggie': 3428,
         'alright': 149, 'funny': 1363, 'chips': 619, 'vinegar': 3442, 'serve': 2790, 'chip': 617, 'brussels': 457,
         'platter': 2359, 'complain': 711, 'plated': 2357, 'beautifully': 326, 'double': 999, 'hummus': 1612,
         'instead': 1685, 'goat': 1411, 'done': 994, 'medium': 1967, 'rare': 2511, 'noisy': 2108, 'horrible': 1593,
         'word': 3574, 'describe': 916, 'neat': 2081, 'light': 1834, 'tops': 3287, 'outdoor': 2204, 'windows': 3551,
         'allowing': 141, 'breeze': 437, 'lively': 1860, 'crowd': 826, 'personally': 2306, 'perhaps': 2300, 'am': 154,
         'reviewing': 2627, 'popped': 2387, 'into': 1695, 'possibly': 2396, 'add': 81, 'star': 3019, 'stay': 3036,
         'superb': 3126, 'eaten': 1059, 'far': 1208, 'who': 3537, 'fan': 1204, 'wrong': 3599, 'yeah': 3606,
         'making': 1921, 'comment': 695, 'stand': 3011, 'statement': 3031, 'crispy': 823, 'golden': 1416,
         'seasoned': 2753, 'perfectly': 2299, 'eat': 1058, 'person': 2302, 'tried': 3320, 'profile': 2455, 'built': 469,
         'heat': 1523, 'sauces': 2718, 'home': 1576, 'speak': 2965, 'deliciousness': 901, 'pickled': 2323,
         'squash': 3002, 'damn': 858, 'barely': 298, 'month': 2040, 'chances': 565, 'wow': 3591, 'getting': 1388,
         'together': 3270, 'takes': 3166, 'always': 153, 'seem': 2769, 'least': 1811, 'understanding': 3370,
         'industry': 1664, 'important': 1637, 'support': 3129, 'local': 1868, 'business': 484, 'friday': 1339,
         'better': 352, 'nicer': 2101, 'meat': 1961, 'actual': 79, 'correctly': 778, 'cooked': 768, 'hard': 1500,
         'real': 2531, 'originally': 2198, 'temp': 3192, 'sent': 2785, 'without': 3563, 'complaint': 714, 'batch': 312,
         'prompt': 2459, 'gave': 1376, 'cheap': 582, 'pour': 2404, '14': 7, 'bourbon': 418, 'notice': 2129,
         'bacon': 276, 'strips': 3085, 'neither': 2092, 'tomato': 3273, 'unless': 3382, 'option': 2189,
         'drenched': 1016, 'dressing': 1019, 'bucket': 460, 'salt': 2697, '75': 41, 'less': 1824, 'please': 2366,
         'window': 3550, 'sits': 2870, 'directly': 963, 'crazy': 812, 'makes': 1920, 'ring': 2640, 'agreed': 120,
         'mention': 1981, 'hoped': 1588, 'sort': 2946, 'straight': 3074, 'vibe': 3436, 'village': 3440, 'credit': 819,
         'knows': 1773, 'its': 1711, 'customer': 847, 'packed': 2235, 'ambience': 159, 'feel': 1233, 'granted': 1438,
         'short': 2831, 'period': 2301, 'apologies': 188, 'repeatedly': 2593, 'ask': 232, 'beverages': 355,
         'disappointed': 967, 'hopes': 1590, 'entree': 1116, 'strictly': 3082, 'small': 2896, 'adults': 97,
         'reason': 2536, 'sad': 2687, 'quite': 2501, 'terrible': 3202, 'simple': 2861, 'miss': 2019, 'bites': 368,
         'grilled': 1454, 'french': 1332, 'safe': 2689, 'understand': 3368, 'mess': 1986, 'dry': 1036, 'bread': 428,
         'bizarre': 371, 'topped': 3284, 'plate': 2356, 'ever': 1137, 'ice': 1621, 'strawberry': 3078, 'vanilla': 3415,
         'mix': 2025, 'scoop': 2738, 'whatever': 3526, 'likely': 1840, 'returning': 2622, 'unfortunate': 3375,
         'specials': 2969, 'groups': 1460, 'especially': 1125, 'warmer': 3478, 'weather': 3505, 'sit': 2868,
         'chicken': 608, 'aioli': 124, 'pros': 2464, 'comfortable': 690, 'fantastic': 1207, 'interesting': 1692,
         'prepared': 2421, 'cons': 739, 'pleasant': 2364, 'unbelievably': 3362, 'kitchen': 1762, 'overall': 2211,
         'man': 1924, 'pleased': 2367, 'left': 1816, 'weekend': 3512, 'slice': 2886, 'toast': 3266, 'slices': 2888,
         'although': 151, 'satisfied': 2712, 'english': 1099, 'muffin': 2055, 'usually': 3410, 'comes': 688,
         'price': 2432, 'mon': 2037, 'additionally': 88, 'walked': 3462, 'craving': 810, 'literally': 1856,
         'happen': 1493, 'included': 1651, 'strange': 3075, 'ham': 1480, 'beets': 336, 'olives': 2168, 'herbs': 1542,
         'wtf': 3601, 'location': 1872, 'lacking': 1779, 'today': 3268, 'lame': 1786, 'knew': 1763, 'her': 1540,
         'recommendation': 2551, 'beyond': 357, 'need': 2084, 'someone': 2931, 'wants': 3475, 'listen': 1853,
         'life': 1833, 'story': 3073, 'awkward': 272, 'poor': 2384, 'fashion': 1210, 'choices': 622, 'having': 1509,
         'aside': 231, 'intense': 1688, 'orange': 2191, 'color': 677, 'smokey': 2907, 'afternoon': 111,
         'lackluster': 1780, 'waitress': 3458, 'seemed': 2770, 'thrilled': 3244, 'fact': 1190, 'anywhere': 184,
         'bed': 331, 'rude': 2671, 'certainly': 558, 'charming': 577, 'spilled': 2985, 'laughed': 1799, 'checking': 587,
         'filling': 1252, 'glasses': 1407, 'etc': 1131, 'generally': 1379, 'realized': 2534, 'planned': 2352,
         'ordering': 2192, 'basket': 310, 'bartenders': 302, 'hung': 1613, 'floor': 1288, 'working': 3580,
         'constantly': 749, 'phones': 2315, 'completely': 719, 'anything': 180, 'related': 2573, 'part': 2256,
         'does': 987, 'prefer': 2414, 'spots': 2996, 'cheers': 591, 'par': 2249, 'cold': 674, 'excellent': 1149,
         'improvement': 1646, 'attentive': 255, 'draw': 1013, 'pricy': 2438, 'worth': 3587, 'seems': 2772, 'avoid': 266,
         'months': 2041, 'ago': 118, 'visited': 3444, 'burgers': 476, 'rush': 2682, 'drinker': 1022, 'showed': 2839,
         'lot': 1884, 'finishing': 1264, 'menus': 1985, 'nearly': 2080, 'empty': 1089, 'continued': 757, 'feet': 1237,
         'stood': 3065, 'chatting': 581, 'row': 2669, 'booths': 403, 'traffic': 3304, 'patient': 2276, 'patron': 2278,
         'world': 3582, 'mark': 1939, '95': 46, 'now': 2131, 'annoyed': 169, 'job': 1722, 'thanks': 3215,
         'confused': 735, 'sorry': 2945, 'looks': 1880, 'sadly': 2688, 'restaurants': 2615, 'works': 3581, 'ain': 123,
         'recently': 2546, 'somewhat': 2935, 'able': 50, 'easily': 1054, 'yelp': 3613, 'lean': 1808, 'savory': 2726,
         'hit': 1565, 'syrup': 3153, 'waffle': 3451, 'waffles': 3452, 'seek': 2768, 'event': 1134, 'schedule': 2736,
         'trip': 3322, 'across': 75, 'quiet': 2500, 'preferred': 2416, 'allowed': 140, 'bf': 358, 'mad': 1911,
         'felt': 1240, 'appropriate': 210, 'season': 2751, 'substitute': 3100, 'slower': 2894, 'toward': 3298,
         'end': 1092, 'inattentive': 1647, 'board': 390, 'rotating': 2661, 'calamari': 501, 'brussel': 456,
         'sprouts': 3000, 'average': 264, 'proceeded': 2449, 'entrees': 1117, 'wings': 3555, 'these': 3224,
         'dishes': 978, 'glad': 1403, 'chorizo': 628, 'quesadilla': 2495, 'suggestions': 3116, 'sampled': 2702,
         'rotation': 2662, 'continue': 756, 'impress': 1640, 'keeping': 1743, 'frequent': 1333, 'celebrate': 549,
         'anniversary': 168, 'gf': 1389, 'hill': 1556, 'dance': 859, 'private': 2443, 'afterwards': 112, 'stop': 3067,
         'closed': 656, 'blocks': 381, 'west': 3524, 'upon': 3396, 'addition': 86, 'providing': 2471, '13': 6,
         'oz': 2229, 'blend': 377, 'la': 1776, 'ground': 1458, 'beef': 332, 'fresh': 1335, 'mushrooms': 2059,
         'red': 2556, 'onions': 2175, 'greens': 1448, 'somewhere': 2936, 'luckily': 1901, 'nicely': 2100, 'juicy': 1734,
         'helped': 1535, 'create': 815, 'sesame': 2798, 'held': 1531, 'notes': 2127, 'accompanying': 68, 'belly': 347,
         'saw': 2727, 'dish': 977, 'green': 1447, 'eggs': 1076, 'thinly': 3233, 'sliced': 2887, 'mustard': 2064,
         'seeds': 2766, 'pop': 2386, 'hash': 1503, 'fruit': 1351, 'palate': 2242, 'inviting': 1700, 'advertised': 103,
         'starting': 3026, '30pm': 26, 'games': 1370, 'began': 339, 'closer': 657, 'incredibly': 1658, 'buffalo': 465,
         'cider': 634, 'honey': 1581, 'start': 3022, 'pig': 2334, 'freshly': 1337, 'covered': 795, 'smoky': 2909,
         'spice': 2979, 'battered': 316, 'whiskey': 3535, 'key': 1747, 'chewy': 604, 'batter': 315, 'coating': 667,
         'fryer': 1356, '17': 10, 'flaky': 1278, 'crisp': 822, 'sharp': 2816, 'provolone': 2472, 'mayo': 1953,
         'moist': 2033, 'lost': 1883, 'decently': 879, 'share': 2813, 'diet': 943, 'case': 534, 'reviews': 2628,
         'adequate': 91, 'hefty': 1530, 'portion': 2391, 'grown': 1462, 'caramelized': 520, 'whipped': 3534,
         'added': 82, 'turned': 3338, 'lol': 1874, 'opted': 2188, 'watering': 3495, 'hole': 1573, 'prior': 2442,
         'interior': 1693, 'lots': 1885, 'wood': 3572, 'natural': 2075, 'lighting': 1836, 'larger': 1790,
         'giving': 1402, 'ample': 166, 'view': 3439, 'impressive': 1643, 'his': 1563, 'personality': 2305,
         'communication': 699, 'personable': 2303, 'mildly': 2000, 'flavors': 1285, 'strawberries': 3077, 'basil': 308,
         'subtle': 3101, 'creative': 818, 'return': 2620, 'delish': 905, 'weeks': 3516, '1st': 13, 'impression': 1642,
         'unexpected': 3374, 'pear': 2288, 'yummy': 3627, 'opened': 2179, 'needs': 2087, 'patty': 2281, 'bun': 472,
         'brown': 454, 'improved': 1645, 'textures': 3209, 'skin': 2881, 'thin': 3227, 'snack': 2912, 'basic': 305,
         'through': 3245, 'ones': 2173, 'bottom': 416, 'bowl': 419, 'worked': 3577, 'kinks': 1761, 'jar': 1719,
         'lights': 1838, 'potential': 2402, 'tender': 3197, 'unhappy': 3378, 'friendliness': 1344, 'goes': 1413,
         'saturday': 2715, 'birthday': 365, 'skeptical': 2877, 'wish': 3560, 'pitcher': 2344, 'deals': 874, 'aren': 216,
         'stuck': 3088, 'morning': 2043, 'filled': 1251, 'champagne': 563, 'buzz': 492, 'overwhelming': 2223,
         'dessert': 932, 'homemade': 1577, 'blt': 387, 'read': 2528, 'description': 918, 'lettuce': 1829, 'weird': 3518,
         'combo': 685, 'itself': 1712, 'impossible': 1639, 'thoroughly': 3236, 'surprised': 3137, 'boy': 423,
         'spicy': 2984, 'approximately': 212, 'tip': 3262, 'considering': 742, 'amount': 164, 'booze': 404,
         'major': 1917, 'props': 2463, 'tall': 3171, 'male': 1922, 'remember': 2584, 'guests': 1469, 'later': 1795,
         'informative': 1670, 'keep': 1742, 'chance': 564, 'presented': 2426, 'simply': 2862, 'win': 3548,
         'prices': 2434, 'dined': 952, 'mother': 2046, 'lady': 1783, 'aka': 127, 'brother': 452, 'smoked': 2906,
         'mom': 2034, 'delectable': 896, 'selections': 2777, 'pumpkin': 2480, 'burrito': 481, 'staple': 3017,
         'classic': 643, 'flavored': 1281, 'pricey': 2435, 'blast': 376, 'soooo': 2943, 'customers': 848, 'alike': 134,
         'random': 2507, 'nobody': 2106, 'waiters': 3456, 'waitresses': 3459, 'doing': 991, 'sign': 2852,
         'waters': 3497, 'mine': 2008, 'consider': 740, 'seeing': 2767, 'future': 1367, 'hoping': 1591, 'control': 760,
         'ourselves': 2203, 'checks': 588, 'casual': 537, 'wide': 3543, 'count': 783, 'longer': 1876, 'glass': 1406,
         'fired': 1266, 'seriously': 2789, 'appetizer': 201, 'various': 3419, 'rating': 2517, 'apps': 213,
         'messy': 1989, 'review': 2623, 'expectations': 1164, 'extra': 1182, 'winter': 3558, 'beet': 335, 'main': 1915,
         'value': 3414, 'welcoming': 3521, 'however': 1605, 'set': 2799, 'apart': 185, 'types': 3348, 'spread': 2997,
         'ranging': 2510, 'tuna': 3335, 'specific': 2971, 'focus': 1295, 'days': 868, 'consumed': 750, 'mixed': 2026,
         'dislike': 979, 'round': 2665, 'conversations': 765, 'style': 3096, 'appearance': 197, 'yesterday': 3617,
         'wasnt': 3486, 'stuff': 3091, 'placed': 2347, 'refills': 2559, 'cant': 517, 'along': 146, 'fall': 1197,
         'type': 3347, 'lemon': 1822, 'punch': 2481, 'goodness': 1420, 'companions': 702, 'between': 353, 'id': 1624,
         'venture': 3430, 'offerings': 2155, 'alone': 145, 'bottle': 413, 'lovers': 1894, 'wise': 3559, 'wife': 3544,
         'tonight': 3279, 'yelpers': 3615, 'issues': 1707, 'look': 1877, 'driving': 1028, 'kudos': 1775, 'whole': 3539,
         'design': 925, '7pm': 42, 'greeted': 1450, 'throughout': 3246, 'pint': 2340, 'power': 2408, 'anyway': 182,
         'pulled': 2479, 'jalapeno': 1715, 'needed': 2085, 'close': 655, 'soggy': 2924, 'folks': 1297, 'em': 1084,
         'proper': 2461, 'blow': 384, 'socks': 2920, 'thinking': 3231, 'above': 51, 'disaster': 971, 'pass': 2263,
         'shells': 2821, 'fully': 1360, 'undercooked': 3365, 'greek': 1446, 'photo': 2316, 'noodle': 2112, 'else': 1082,
         'presentation': 2425, 'romaine': 2655, 'leaf': 1807, 'pile': 2335, 'middle': 1996, 'soaked': 2918,
         'cubes': 834, 'feta': 1242, 'kidding': 1751, 'dollar': 992, 'cut': 850, 'tomatoes': 3274, 'tastier': 3181,
         'bell': 346, 'pepper': 2293, 'chef': 599, 'owner': 2226, 'clear': 651, 'ton': 3276, 'watched': 3491,
         'assumed': 242, 'strong': 3086, 'competition': 710, 'afford': 107, 'work': 3576, 'near': 2078, '2nd': 24,
         'forgotten': 1315, 'tastes': 3180, 'spent': 2978, 'total': 3291, 'coconut': 670, 'curry': 845,
         'memorable': 1977, 'item': 1709, 'flavorful': 1282, 'dip': 959, 'gets': 1387, 'sometimes': 2934,
         'difficult': 948, 'tough': 3295, 'majority': 1918, 'reservations': 2606, 'setup': 2804, 'beautiful': 325,
         'summer': 3120, 'narrow': 2072, 'pie': 2330, 'gravy': 1441, 'ugh': 3352, 'clean': 646, 'omelette': 2170,
         'spinach': 2986, 'hands': 1488, 'bay': 317, 'seasoning': 2754, 'year': 3607, 'sister': 2867, 'wonderful': 3568,
         'heck': 1529, 'yea': 3605, 'rarely': 2512, 'philadelphia': 2311, 'managed': 1926, 'kick': 1748, 'tv': 3341,
         'aged': 116, 'seen': 2773, 'kicked': 1749, 'changing': 569, 'gift': 1391, 'card': 521, 'parm': 2254,
         'white': 3536, 'terrific': 3204, 'book': 399, 'passed': 2265, 'bright': 442, 'dim': 950, 'fire': 1265,
         'overwhelmed': 2222, '15': 8, 'executed': 1158, 'except': 1150, 'drank': 1012, 'fashioned': 1211,
         'cherry': 601, 'rittenhouse': 2645, 'rye': 2686, 'cannot': 516, 'kinda': 1757, 'known': 1772, 'parties': 2259,
         'space': 2959, 'designed': 926, 'serious': 2788, 'growing': 1461, 'numerous': 2134, 'forth': 1319,
         'basis': 309, 'score': 2739, 'bold': 393, 'likes': 1841, 'offers': 2156, 'ingredients': 1674,
         'promising': 2458, 'deep': 888, 'reach': 2526, 'temperature': 3193, 'meanwhile': 1960, 'failed': 1193,
         'sense': 2784, 'blue': 388, 'fit': 1271, 'fat': 1214, 'brings': 445, 'thumbs': 3251, 'haven': 1508,
         'partner': 2260, 'pocket': 2375, 'track': 3301, 'doesn': 988, 'wednesday': 3508, 'girlfriend': 1396,
         'cute': 851, 'tucked': 3333, 'discovered': 974, 'oh': 2160, 'decor': 885, 'bottles': 415, 'normal': 2118,
         'italian': 1708, 'shared': 2814, 'helpful': 1536, 'knowledgeable': 1771, 'became': 327, 'nervous': 2093,
         'waitstaff': 3460, 'training': 3307, 'former': 1318, 'john': 1724, 'bbq': 318, 'saucy': 2719, 'martini': 1941,
         'fiance': 1244, 'hope': 1587, 'mins': 2013, 'falls': 1199, 'missing': 2021, 'ours': 2202, 'besides': 350,
         'issue': 1706, 'cheesy': 598, 'appreciated': 207, 'recommended': 2553, 'pot': 2399, 'sunny': 3124,
         'underwhelming': 3373, 'boring': 406, 'greatest': 1444, 'afraid': 109, 'heart': 1521, 'pub': 2475,
         'ordinary': 2194, 'fare': 1209, 'tvs': 3342, 'cafe': 497, 'watching': 3492, 'cozy': 800, 'venue': 3432,
         'ability': 49, 'walk': 3461, 'bars': 300, 'grab': 1428, 'thanksgiving': 3216, 'ways': 3501, 'seat': 2756,
         'upside': 3401, 'efficient': 1072, 'show': 2838, 'chocolate': 620, 'dozen': 1009, 'loves': 1895,
         'fingers': 1261, 'clearly': 653, 'scratch': 2741, 'downside': 1006, 'breading': 430, 'soon': 2940,
         'easy': 1057, 'fix': 1274, 'difference': 945, 'dressed': 1018, 'lime': 1844, 'bucks': 461, 'glaze': 1408,
         'mush': 2057, 'ball': 286, 'ketchup': 1746, 'standard': 3012, 'highlight': 1553, 'inventive': 1698,
         'fry': 1355, 'maple': 1934, 'expect': 1162, 'pay': 2282, 'dig': 949, 'mainly': 1916, 'reasonably': 2538,
         'priced': 2433, 'surprise': 3136, 'thick': 3225, 'cheaper': 583, 'meals': 1955, 'boyfriend': 424,
         'respect': 2610, 'justice': 1739, 'descriptions': 919, 'outstanding': 2208, 'hadn': 1474, 'trendy': 3318,
         'crunchy': 830, 'weight': 3517, 'slight': 2890, 'guessing': 1467, 'filet': 1249, 'content': 755,
         'squeezed': 3004, 'pink': 2339, 'juices': 1733, 'complete': 718, 'finish': 1262, 'rib': 2630, 'taco': 3158,
         'sounded': 2950, 'somehow': 2930, 'tacos': 3159, 'mini': 2009, 'baked': 282, 'tortilla': 3289, 'highly': 1555,
         'waste': 3487, 'believe': 345, 'ambiance': 158, 'mid': 1995, 'understaffed': 3367, 'hold': 1570, 'crab': 801,
         'cake': 499, 'cause': 545, 'blame': 374, 'refill': 2557, 'watch': 3490, 'gorgeous': 1424, 'classics': 644,
         'attempt': 250, 'meh': 1971, 'manhattan': 1931, 'balance': 284, 'hers': 1543, 'hopefully': 1589,
         'forget': 1312, 'extraordinary': 1183, 'overpriced': 2220, 'mediocre': 1966, 'child': 609, 'spectacular': 2973,
         'accommodate': 63, 'twenty': 3343, 'scrambled': 2740, 'tell': 3189, 'taking': 3167, 'complained': 712,
         'starving': 3028, 'apologetic': 187, 'aware': 267, 'kept': 1745, 'cup': 840, 'overpowering': 2219,
         'awful': 270, 'gracious': 1432, '21': 16, 'stir': 3059, 'thankfully': 3214, 'within': 3562, 'arriving': 226,
         'shouldn': 2836, 'woman': 3564, 'son': 2937, 'mean': 1956, 'yourself': 3624, 'de': 870, 'knowing': 1768,
         'nuts': 2136, 'recommendations': 2552, 'husband': 1619, 'picky': 2326, 'pleasantly': 2365, 'piece': 2331,
         'nonetheless': 2111, 'annoying': 170, 'fluffy': 1292, 'walnut': 3468, 'caramel': 519, 'carte': 533,
         'disappointing': 968, 'roasted': 2648, 'already': 148, 'piping': 2341, 'locals': 1870, 'moving': 2051,
         'stopping': 3069, 'frequently': 1334, 'march': 1935, 'fans': 1206, 'bonus': 397, 'line': 1847,
         'attention': 254, 'app': 192, 'suddenly': 3110, 'degrees': 894, 'severely': 2807, 'care': 524, 'frozen': 1350,
         'throwing': 3248, 'paid': 2237, 'low': 1897, 'solo': 2928, 'mission': 2022, 'positive': 2394, 'initial': 1675,
         'certain': 557, 'occupied': 2147, 'flow': 1291, 'chair': 560, 'varieties': 3417, 'underwhelmed': 3372,
         'expected': 1165, 'runs': 2681, 'overdone': 2214, 'nor': 2116, 'forgettable': 1313, 'delivered': 907,
         'third': 3234, 'lovely': 1892, 'drinking': 1023, 'returned': 2621, 'appeared': 198, 'product': 2452,
         'opens': 2181, 'fills': 1253, 'serves': 2794, 'poured': 2405, 'politely': 2382, 'dreams': 1015,
         'anticipated': 175, 'response': 2612, 'use': 3405, 'assume': 241, 'unable': 3357, 'frankly': 1326,
         'settle': 2802, 'move': 2048, 'reserved': 2608, 'comments': 697, 'mistake': 2023, 'common': 698,
         'unusual': 3391, 'worst': 3586, 'lover': 1893, 'act': 76, 'visits': 3446, 'significant': 2854,
         'destination': 934, 'level': 1830, 'indeed': 1659, 'served': 2791, 'step': 3049, 'change': 566,
         'consistently': 746, 'bother': 411, 'pic': 2318, 'posted': 2398, 'improve': 1644, 'hitting': 1567,
         'behind': 343, 'chairs': 561, 'none': 2110, 'horrendous': 1592, '90': 45, 'gross': 1457, 'liking': 1842,
         'edible': 1069, 'choose': 623, 'nj': 2105, 'coast': 664, 'effort': 1073, 'grade': 1433, 'guest': 1468,
         'dirty': 964, 'napkins': 2071, 'picked': 2320, 'station': 3034, 'wet': 3525, 'cleaned': 647, 'promptly': 2460,
         'appear': 196, 'entirely': 1114, 'aspect': 237, 'occasions': 2146, 'knowledge': 1770, 'sells': 2781,
         'help': 1534, 'appetizers': 202, 'appreciate': 206, 'pancakes': 2245, 'yum': 3626, 'cheeseburger': 593,
         'recommend': 2550, 'anyone': 179, 'generic': 1380, 'peach': 2285, 'candle': 513, 'women': 3565,
         'bathroom': 313, 'definite': 891, 'desired': 928, 'standing': 3014, 'closing': 660, 'break': 432, 'self': 2778,
         'stopped': 3068, 'intended': 1687, 'resist': 2609, 'burned': 478, 'pita': 2343, 'generous': 1381,
         'hubby': 1608, 'greasy': 1443, 'paying': 2283, 'walking': 3463, 'distance': 981, 'lightly': 1837,
         'perfection': 2298, 'lighter': 1835, 'version': 3433, 'broth': 451, 'overlooked': 2216, 'deserve': 922,
         'separate': 2786, 'seven': 2805, 'talked': 3169, 'grew': 1452, 'currently': 844, 'god': 1412, 'till': 3257,
         'kid': 1750, 'born': 407, 'rings': 2641, 'true': 3327, 'note': 2125, 'weekends': 3513, 'refreshing': 2560,
         'japanese': 1718, 'box': 421, 'bigger': 361, 'depth': 915, 'feature': 1225, 'traditional': 3303, 'plain': 2350,
         'butter': 488, 'garlic': 1373, 'olive': 2167, 'oil': 2161, 'grill': 1453, 'flakes': 1277, 'indian': 1660,
         'eastern': 1056, 'absolute': 52, 'fair': 1194, 'judging': 1731, 'general': 1378, 'clientele': 654,
         'cater': 541, 'families': 1201, 'children': 610, 'older': 2166, 'couples': 788, 'expensive': 1167, 'airy': 126,
         'football': 1308, 'comfortably': 691, 'connoisseur': 738, 'sip': 2866, 'shift': 2822, 'thank': 3212,
         'suggest': 3113, 'sticking': 3054, 'cravings': 811, 'ratings': 2518, 'appealing': 195, 'luck': 1900,
         'surprisingly': 3139, 'arugula': 228, 'contained': 752, 'bits': 369, 'teeth': 3188, 'gluten': 1410,
         'allergy': 137, 'replacement': 2596, 'catch': 538, 'fell': 1238, 'ignore': 1628, 'fork': 1316,
         'moderately': 2031, 'meet': 1968, 'dietary': 944, 'free': 1329, 'management': 1927, 'cook': 767,
         'cleaning': 649, 'prep': 2418, 'cross': 825, 'breast': 435, 'veggies': 3429, 'hell': 1532, 'dog': 989,
         'shit': 2824, 'mmmm': 2030, 'convinced': 766, 'bone': 395, 'overpowered': 2218, 'seared': 2750, 'duck': 1037,
         'ginger': 1394, 'vegetables': 3425, 'stew': 3052, 'consistent': 745, 'sea': 2746, 'bass': 311, 'massive': 1946,
         'eclectic': 1066, 'elsewhere': 1083, 'building': 468, 'years': 3608, 'walls': 3467, 'latest': 1796,
         'favorites': 1222, 'eager': 1046, 'sugar': 3111, 'allow': 139, 'matter': 1948, 'polite': 2381,
         'described': 917, 'section': 2764, 'mind': 2007, '25': 20, 'nope': 2115, 'thru': 3250, 'swing': 3150,
         'correct': 777, 'slightly': 2891, 'joke': 1728, 'talking': 3170, 'presence': 2423, 'pissed': 2342,
         'joint': 1726, 'typically': 3350, 'healthier': 1516, 'thrown': 3249, 'regulars': 2571, 'cooking': 771,
         'favor': 1220, 'establishment': 1129, 'handful': 1484, 'eh': 1077, 'flying': 1294, 'speed': 2974,
         'focused': 1296, 'showing': 2840, 'coated': 666, 'dipping': 961, 'octopus': 2148, 'gripe': 1455,
         'standout': 3015, 'surrounding': 3141, 'running': 2679, 'must': 2063, 'acknowledged': 74, 'employees': 1088,
         'lack': 1777, 'pizza': 2345, 'write': 3596, 'non': 2109, 'stupid': 3095, 'unnecessary': 3384, 'crap': 807,
         'silverware': 2858, 'refilled': 2558, 'cooks': 772, 'smoke': 2905, 'ridiculous': 2637, 'honest': 1579,
         'feedback': 1232, 'die': 942, 'often': 2159, 'chill': 612, 'spacious': 2960, 'eagles': 1047, 'playing': 2363,
         'tasteless': 3179, 'wawa': 3499, 'skip': 2882, 'vegetable': 3424, 'overcooked': 2213, 'pasta': 2269,
         'late': 1793, 'samples': 2704, 'question': 2496, 'asking': 234, 'flag': 1276, 'deliver': 906,
         'particularly': 2258, 'changes': 568, 'five': 1273, 'apartment': 186, 'bunch': 473, 'phenomenal': 2309,
         'dad': 855, 'stock': 3060, 'advertise': 102, 'accommodating': 65, 'terms': 3201, 'adding': 85, 'band': 291,
         'enjoyable': 1101, 'black': 372, 'cents': 556, 'operation': 2182, 'brand': 427, 'eateries': 1061,
         'settled': 2803, 'nights': 2104, 'healthy': 1517, 'paired': 2240, 'unique': 3380, 'knowledgable': 1769,
         'keeps': 1744, 'bagel': 280, 'forgot': 1314, 'moments': 2036, 'shortly': 2832, 'assuming': 243, 'cancel': 511,
         'law': 1801, 'minor': 2012, 'seats': 2759, 'block': 380, 'christmas': 631, 'decorated': 886, 'standards': 3013,
         'figure': 1247, 'guess': 1466, 'countless': 785, 'wrote': 3600, 'worthy': 3588, 'seemingly': 2771,
         'endless': 1095, 'hits': 1566, 'amongst': 163, 'hop': 1586, 'admit': 92, 'depending': 913, 'anytime': 181,
         'hate': 1506, 'reading': 2529, 'dear': 875, 'stuffed': 3092, 'saying': 2729, 'freaking': 1328, 'grabbed': 1429,
         'involved': 1701, 'rounded': 2666, 'gotten': 1426, 'becoming': 330, 'arrival': 222, 'checked': 586,
         'offering': 2154, 'discount': 972, 'beverage': 354, 'refused': 2563, 'lie': 1832, 'suck': 3106, 'eve': 1132,
         'tons': 3280, 'supposed': 3132, 'uh': 3353, 'gigantic': 1392, 'mistakes': 2024, 'apparently': 193,
         'means': 1958, 'zero': 3628, 'paper': 2248, 'clue': 662, 'train': 3305, 'hire': 1562, 'apologized': 190,
         'explanation': 1175, 'problems': 2448, 'status': 3035, 'handled': 1486, 'celebrating': 550, 'trek': 3317,
         'stayed': 3037, 'blah': 373, 'scene': 2735, 'chain': 559, 'pricier': 2436, 'original': 2197, 'calling': 505,
         'switch': 3151, 'due': 1039, 'chipotle': 618, 'omg': 2171, 'chunky': 633, 'pm': 2373, 'rounds': 2667,
         'themed': 3220, 'multiple': 2056, 'dark': 862, 'avocado': 265, 'complimentary': 722, 'ticket': 3254,
         'unacceptable': 3358, 'met': 1990, 'moved': 2049, 'girls': 1398, 'trouble': 3325, 'um': 3355, 'esque': 1127,
         'chopped': 626, 'eater': 1060, 'stated': 3030, 'plan': 2351, 'popular': 2389, 'pick': 2319, 'typical': 3349,
         'diner': 953, 'class': 642, 'holiday': 1574, 'admittedly': 93, 'coke': 673, 'wanting': 3474, 'sub': 3097,
         'essentially': 1128, 'shredded': 2842, 'bag': 279, 'noise': 2107, 'questions': 2497, 'against': 114,
         'stale': 3010, 'brick': 440, 'exterior': 1181, 'compliments': 724, 'creating': 817, 'wheat': 3528,
         'chose': 629, 'exceptional': 1152, 'citrus': 637, 'bursting': 482, 'tangy': 3172, 'almond': 143,
         'accompanied': 66, 'courses': 791, 'silly': 2857, 'cheesecake': 594, 'apples': 205, 'sweetness': 3147,
         'complimented': 723, 'wonderfully': 3569, 'delighted': 903, 'reminded': 2587, 'pound': 2403, 'salted': 2698,
         'visiting': 3445, 'informed': 1671, 'complaining': 713, 'possible': 2395, 'microwaved': 1994, 'bc': 319,
         'confirmed': 734, 'via': 3435, 'account': 71, 'brisket': 446, 'carry': 531, 'kids': 1752, 'alcohol': 131,
         'margarita': 1936, 'charged': 573, 'removed': 2591, 'acceptable': 57, 'thursday': 3252, 'hanging': 1490,
         'wifi': 3545, 'counter': 784, 'fave': 1219, 'date': 864, 'diverse': 984, 'lose': 1881, 'latter': 1797,
         'tends': 3199, 'hipster': 1561, 'parents': 2250, 'letting': 1828, 'leaves': 1813, 'rip': 2642,
         'combination': 682, 'lacked': 1778, 'canned': 515, 'screw': 2744, 'screaming': 2742, 'soft': 2923,
         'sautéed': 2722, 'beans': 321, 'salsa': 2696, 'listed': 1852, 'vast': 3420, 'immediately': 1634,
         'fourth': 1325, 'newly': 2097, 'opinion': 2183, 'opportunity': 2184, 'decision': 883, 'sandwiches': 2708,
         'foods': 1304, 'regardless': 2565, 'hair': 1476, 'changed': 567, 'microwave': 1993, 'pack': 2232, '100': 2,
         'save': 2723, 'cuisine': 837, 'slaw': 2884, 'rid': 2635, 'heaven': 1525, 'roommate': 2658, 'worried': 3583,
         'awkwardly': 273, 'suppose': 3131, 'fault': 1217, 'center': 554, 'apologize': 189, 'inedible': 1665,
         'trust': 3330, 'sucked': 3107, 'features': 1227, 'odd': 2149, 'closest': 659, 'cauliflower': 544, 'laid': 1784,
         'dried': 1020, 'turkey': 3336, 'imagine': 1632, 'juice': 1732, 'environment': 1119, 'themselves': 3221,
         'biased': 359, 'test': 3205, 'pricing': 2437, 'dollars': 993, 'personal': 2304, 'con': 725, 'mushy': 2060,
         'char': 570, 'worker': 3578, 'ignored': 1629, 'glance': 1405, 'sharing': 2815, 'sidewalk': 2849, 'below': 348,
         'array': 221, '40': 30, 'hamburger': 1481, 'oven': 2209, '5th': 37, 'background': 275, 'ensure': 1107,
         'notch': 2124, 'signature': 2853, 'balanced': 285, 'puts': 2489, 'slide': 2889, 'stingy': 3058, 'called': 504,
         'reserve': 2607, 'ahead': 122, 'spoke': 2992, 'staring': 3020, 'phone': 2314, 'booth': 402, 'member': 1975,
         'speaks': 2967, 'telling': 3190, 'apology': 191, 'inconvenience': 1655, 'accordingly': 70, 'feelings': 1235,
         'understood': 3371, 'trained': 3306, 'fail': 1192, 'provide': 2468, 'timely': 3258, 'plentiful': 2370,
         'obnoxious': 2139, 'beginning': 341, 'iced': 1622, 'rubbery': 2670, 'watered': 3494, 'boiled': 392,
         'passable': 2264, 'ya': 3603, 'borderline': 405, 'rushing': 2684, 'liquor': 1850, 'overbearing': 2212,
         'tab': 3155, 'updated': 3394, 'cares': 528, 'tenders': 3198, 'squeeze': 3003, 'salty': 2699, 'tax': 3184,
         'taken': 3164, 'shock': 2826, 'eggplant': 1075, 'bean': 320, 'six': 2873, 'handed': 1483, 'town': 3300,
         'treated': 3314, 'poorly': 2385, 'unfriendly': 3377, 'rushed': 2683, 'desserts': 933, 'pointed': 2377,
         'placing': 2349, 'suggested': 3114, 'fabulous': 1187, 'owners': 2227, 'weeknight': 3515, 'usual': 3409,
         'send': 2783, 'entered': 1109, 'desk': 929, 'burning': 479, 'declined': 884, 'dropped': 1032,
         'significantly': 2855, 'overheard': 2215, 'fiancé': 1245, 'king': 1760, 'whim': 3533, 'ill': 1630,
         'smell': 2899, 'direction': 962, 'specialty': 2970, 'chefs': 600, 'shows': 2841, 'higher': 1551, 'movie': 2050,
         'asparagus': 236, 'thicker': 3226, 'shall': 2809, 'edit': 1070, 'tequila': 3200, 'edge': 1067,
         'outrageous': 2206, 'hood': 1583, 'appropriately': 211, 'blowing': 385, 'pesto': 2307, 'balsamic': 288,
         'ribs': 2631, 'relatively': 2574, 'mouth': 2047, 'dogs': 990, 'fav': 1218, 'cuz': 854, 'treat': 3313,
         'broad': 447, 'includes': 1652, 'flight': 1287, 'number': 2133, 'seconds': 2762, 'shopping': 2829,
         'cabbage': 495, 'combined': 684, 'girl': 1395, 'burn': 477, 'calories': 508, 'finding': 1257, 'legs': 1821,
         'hang': 1489, 'amazingly': 157, 'companion': 701, 'age': 115, 'range': 2509, 'realize': 2533,
         'necessarily': 2082, 'sounds': 2952, 'preparation': 2419, 'modern': 2032, 'final': 1254, 'dont': 995,
         'putting': 2490, 'wonder': 3567, 'crack': 802, 'turns': 3340, 'practically': 2410, 'thats': 3217,
         'properly': 2462, 'heavy': 1528, 'family': 1202, 'explain': 1172, 'comped': 708, 'beat': 322, 'tofu': 3269,
         'smooth': 2910, 'store': 3071, 'bought': 417, 'covid': 796, 'vibes': 3437, 'burnt': 480, 'answer': 172,
         'quantity': 2493, 'result': 2619, 'ridiculously': 2638, 'greet': 1449, 'meaning': 1957, 'receive': 2542,
         'albeit': 130, 'error': 1123, 'message': 1987, 'female': 1241, 'fill': 1250, 'thus': 3253, 'occasion': 2143,
         'cell': 553, 'relaxed': 2576, 'knock': 1765, 'selected': 2775, 'welcome': 3519, 'vodka': 3447, 'highest': 1552,
         'container': 753, 'wrapped': 3594, 'plastic': 2355, 'tight': 3255, 'workers': 3579, 'wrap': 3593,
         'throw': 3247, 'pool': 2383, 'patio': 2277, 'trick': 3319, '16': 9, '35': 27, 'southern': 2957, 'concept': 726,
         'rule': 2675, 'intrigued': 1696, 'opposite': 2186, 'amounts': 165, 'ends': 1096, 'reasons': 2539,
         'altogether': 152, 'mashed': 1943, 'broccoli': 448, 'sum': 3118, '18': 11, 'drop': 1031, 'substantial': 3099,
         'crushed': 831, 'shell': 2820, 'ps': 2474, 'upper': 3397, 'scale': 2731, 'snag': 2914, 'feels': 1236,
         'answered': 173, 'gem': 1377, 'creme': 820, 'lived': 1859, 'jam': 1717, 'understandable': 3369, 'compare': 705,
         'reminds': 2588, '26': 21, 'lets': 1827, 'eye': 1185, 'contact': 751, 'sell': 2779, 'drunk': 1035,
         'regularly': 2570, 'stops': 3070, 'degree': 893, 'fits': 1272, 'concerned': 728, 'introduced': 1697,
         'twist': 3345, 'convenient': 762, 'towards': 3299, 'spending': 2977, 'report': 2598, 'additional': 87,
         'oddly': 2150, 'dump': 1040, 'onto': 2177, 'rice': 2632, 'skimpy': 2880, 'hoagie': 1568, 'charred': 578,
         'mushroom': 2058, 'advice': 104, 'milk': 2003, 'basics': 307, 'lit': 1855, 'google': 1423, 'states': 3032,
         'noon': 2114, 'state': 3029, 'knocked': 1766, 'ratio': 2519, 'occasional': 2144, 'character': 571,
         'headed': 1512, 'missed': 2020, 'lukewarm': 1904, 'neighbors': 2091, 'repeat': 2592, 'insanely': 1679,
         'route': 2668, 'lately': 1794, 'entrée': 1118, '3rd': 29, 'incredible': 1657, 'example': 1147,
         'suggestion': 3115, 'meant': 1959, 'happier': 1497, 'btw': 458, 'unsure': 3389, 'bus': 483, 'weekday': 3510,
         'sized': 2875, 'obvious': 2141, 'lo': 1864, 'incorrect': 1656, 'address': 89, 'advised': 106, 'advantage': 99,
         'worry': 3584, 'claim': 639, 'email': 1085, 'regards': 2566, 'miserable': 2018, 'succulent': 3104,
         'recall': 2540, 'chew': 603, 'tongue': 3278, 'secret': 2763, 'truth': 3331, 'website': 3506, 'nicest': 2102,
         'equally': 1122, 'conclusion': 730, 'becomes': 329, 'ha': 1473, 'meeting': 1969, 'per': 2296, 'costs': 780,
         'warned': 3480, 'diced': 938, 'mint': 2014, 'recipe': 2547, 'curious': 842, 'ages': 117, 'bathrooms': 314,
         'cheesesteak': 596, 'whatsoever': 3527, 'min': 2006, 'replied': 2597, '55': 35, 'grease': 1442,
         'swimming': 3149, 'co': 663, 'holding': 1571, 'gentleman': 1384, 'tells': 3191, 'exist': 1160, 'page': 2236,
         'employee': 1087, 'baby': 274, 'angry': 167, 'execution': 1159, 'hip': 1560, 'college': 676, 'chicago': 606,
         'weekdays': 3511, 'travel': 3309, 'areas': 215, 'tourist': 3297, 'unimpressed': 3379, 'package': 2233,
         'mall': 1923, 'hurt': 1618, 'picture': 2328, 'student': 3089, 'texas': 3206, 'fool': 1305, 'fooled': 1306,
         'loving': 1896, 'décor': 1044, 'bothered': 412, 'occasionally': 2145, 'attitude': 256, 'deliciously': 900,
         'spare': 2963, 'disappoint': 966, 'follow': 1298, 'instagram': 1683, 'photos': 2317, '4th': 33, 'july': 1735,
         'delivery': 910, 'grubhub': 1464, 'inexpensive': 1666, 'gladly': 1404, 'hidden': 1549, 'created': 816,
         'finger': 1260, 'soda': 2921, 'road': 2646, 'lines': 1848, 'wont': 3571, 'traveled': 3310, 'locally': 1869,
         'bottled': 414, 'wines': 3553, 'surely': 3135, 'winner': 3556, 'cramped': 805, 'seafood': 2747, 'desert': 920,
         'til': 3256, 'yelling': 3611, 'matters': 1949, 'messed': 1988, 'sports': 2994, 'earth': 1051, 'accompany': 67,
         'whenever': 3529, 'join': 1725, 'tomorrow': 3275, 'explained': 1173, 'unlike': 3383, 'dine': 951,
         'handle': 1485, 'foot': 1307, 'caught': 543, 'nose': 2123, 'shocked': 2827, 'disappeared': 965, 'raw': 2524,
         'icing': 1623, 'receiving': 2544, 'def': 889, 'interaction': 1689, 'rain': 2502, 'followed': 1299,
         'request': 2600, '60': 38, 'unappetizing': 3359, 'chic': 605, 'patronize': 2279, 'led': 1815, 'root': 2659,
         'grocery': 1456, 'portioned': 2392, 'breaded': 429, 'equal': 1121, 'genuinely': 1386, 'cleared': 652, '00': 0,
         'satisfy': 2713, 'signs': 2856, 'choosing': 624, 'consistency': 744, 'knife': 1764, 'flavorless': 1284,
         'steak': 3040, 'inch': 1648, 'insult': 1686, 'reality': 2532, 'dream': 1014, 'himself': 1558, 'fixed': 1275,
         '27': 22, 'bustling': 486, 'lucky': 1902, 'com': 681, 'ceiling': 548, 'layout': 1804, 'rustic': 2685,
         'wall': 3465, 'trips': 3324, 'chat': 579, 'establishments': 1130, 'sound': 2949, 'enter': 1108,
         'unpretentious': 3386, 'wind': 3549, 'provided': 2469, 'aroma': 219, 'acknowledge': 73, 'braised': 426,
         'complemented': 717, 'upscale': 3399, 'subpar': 3098, 'falling': 1198, 'adds': 90, 'expert': 1171,
         'remaining': 2581, 'comparable': 704, 'immediate': 1633, 'imo': 1635, 'heads': 1514, 'accept': 56,
         'sight': 2851, 'reviewers': 2626, 'sweeter': 3146, 'indoor': 1662, 'rose': 2660, 'begin': 340, 'sigh': 2850,
         'sold': 2925, 'lives': 1861, 'ish': 1703, 'sampling': 2705, 'joints': 1727, 'downtown': 1008,
         'organized': 2196, 'tour': 3296, 'search': 2748, 'watermelon': 3496, 'anymore': 178, 'fatty': 1216,
         'lousy': 1889, 'sucks': 3109, 'cover': 794, 'news': 2098, 'hasn': 1504, 'dead': 871, 'underneath': 3366,
         'touched': 3294, 'register': 2567, 'health': 1515, 'goods': 1421, 'seasonal': 2752, 'regret': 2568,
         'restroom': 2617, 'differently': 947, 'doors': 998, 'hall': 1479, 'colors': 680, 'stumbled': 3094,
         'classy': 645, 'requests': 2602, '9pm': 48, 'wanna': 3471, 'comfy': 693, 'yo': 3619, 'winning': 3557,
         'miles': 2002, 'hotel': 1600, 'produce': 2451, 'provides': 2470, 'mild': 1999, 'happening': 1495, 'fee': 1230,
         'grub': 1463, 'drive': 1026, 'office': 2157, 'bargain': 299, 'staples': 3018, 'alternative': 150,
         'voice': 3448, 'jumped': 1738, 'deserved': 923, 'factor': 1191, 'dying': 1043, 'raving': 2522, 'opt': 2187,
         'marinated': 1938, 'wished': 3561, 'sides': 2848, 'protein': 2466, 'sizes': 2876, 'iron': 1702, 'cost': 779,
         'grain': 1434, 'melted': 1973, 'herb': 1541, 'organic': 2195, 'buffet': 466, 'carrot': 529, 'satisfying': 2714,
         'sodas': 2922, 'pre': 2413, 'pride': 2439, 'owned': 2225, 'shop': 2828, 'nut': 2135, 'soy': 2958, 'al': 128,
         'names': 2069, 'memory': 1979, 'cafeteria': 498, 'kale': 1741, 'tray': 3312, 'stellar': 3048, 'oily': 2162,
         'dairy': 857, 'sooo': 2942, 'million': 2005, 'freshest': 1336, 'writing': 3597, 'trash': 3308, 'garden': 1372,
         'base': 303, 'corn': 775, 'heavenly': 1526, 'peppers': 2295, 'coworker': 798, 'agree': 119, 'chick': 607,
         'peas': 2289, 'customize': 849, 'bang': 292, 'buck': 459, 'vegetarians': 3427, 'beware': 356, 'shrimp': 2843,
         'ranch': 2506, 'market': 1940, 'pace': 2231, 'include': 1650, 'suspect': 3143, 'everyday': 1140, 'risk': 2644,
         'affordable': 108, 'gives': 1401, 'thoughtful': 3240, 'accessible': 60, 'vegans': 3423, 'whoever': 3538,
         'colorful': 679, 'team': 3186, 'zucchini': 3630, 'allergic': 135, 'recipes': 2548, 'restrictions': 2616,
         'pair': 2239, 'happily': 1498, 'picking': 2321, 'buy': 490, 'pretentious': 2428, 'spanish': 2962, 'beats': 324,
         'legit': 1820, 'wallet': 3466, 'contrast': 759, 'vinaigrette': 3441, 'square': 3001, 'pull': 2478,
         'comparison': 707, '99': 47, 'melts': 1974, 'raved': 2521, 'thai': 3210, 'celery': 552, 'korean': 1774,
         'freshness': 1338, 'rich': 2633, 'suit': 3117, 'meats': 1964, 'eaters': 1062, 'site': 2869, 'daily': 856,
         'noodles': 2113, 'carrots': 530, 'spiced': 2980, 'following': 1300, 'outdoors': 2205, 'shake': 2808,
         'eatery': 1063, 'process': 2450, 'accidentally': 62, 'theme': 3219, 'cranberry': 806, 'killer': 1755,
         'confusing': 736, 'calm': 507, 'unbelievable': 3361, 'swear': 3144, 'justify': 1740, 'careful': 526,
         'melt': 1972, 'treats': 3316, 'allergies': 136, 'featuring': 1228, 'steep': 3046, 'utensils': 3411,
         'upstairs': 3402, 'wondering': 3570, '19': 12, 'appeal': 194, 'preference': 2415, 'breaking': 434, 'bank': 294,
         'community': 700, 'adorable': 94, 'gotta': 1425, 'innovative': 1677, 'relax': 2575, 'weak': 3502,
         'pineapple': 2338, 'veg': 3421, 'buying': 491, 'inconsistent': 1654, 'quarter': 2494, '4pm': 32,
         'written': 3598, 'realizing': 2535, 'distinct': 982, 'stomach': 3063, 'stays': 3039, 'desperate': 930,
         'mentioning': 1983, 'pure': 2484, 'faces': 1189, 'tips': 3264, 'catering': 542, 'cards': 523, 'familiar': 1200,
         'balls': 287, 'located': 1871, 'appetizing': 203, 'hooked': 1585, 'online': 2176, 'crappy': 808,
         'popping': 2388, 'express': 1178, 'delightful': 904, 'tossed': 3290, 'heavily': 1527, 'crunch': 829,
         'stocked': 3061, 'traveling': 3311, 'norm': 2117, 'learned': 1810, 'servings': 2797, 'faster': 1213,
         'success': 3102, 'guilty': 1470, 'pictures': 2329, 'alley': 138, 'varied': 3416, 'wandered': 3469,
         'platters': 2360, 'nyc': 2138, 'cashier': 536, 'moment': 2035, 'named': 2068, 'parts': 2261, 'anyways': 183,
         'members': 1976, 'wearing': 3504, 'select': 2774, 'lunchtime': 1907, 'cheapest': 584, 'professional': 2454,
         'tho': 3235, 'amazed': 155, 'everywhere': 1144, 'epic': 1120, 'encourage': 1091, 'wooden': 3573,
         'manage': 1925, 'pathetic': 2274, 'pa': 2230, 'fellow': 1239, 'yelper': 3614, '80': 43, 'wedding': 3507,
         'mexican': 1992, 'boxes': 422, 'leftovers': 1818, 'allows': 142, 'previously': 2431, 'dc': 869, 'mixing': 2027,
         'layer': 1802, 'disgusting': 976, 'crafted': 804, 'delivering': 908, 'containers': 754, 'tipped': 3263,
         'searching': 2749, 'charging': 575, 'using': 3408, 'phila': 2310, 'homey': 1578, 'men': 1980, 'helping': 1537,
         'yay': 3604, 'assortment': 240, 'adventurous': 101, 'services': 2795, 'preparing': 2422, 'authentic': 258,
         'welcomed': 3520, 'combos': 686, 'joe': 1723, 'custom': 846, 'courteous': 792, 'successful': 3103,
         'wandering': 3470, 'holy': 1575, 'cow': 797, 'feed': 1231, 'supermarket': 3128, 'deli': 897, 'fountain': 1323,
         'acted': 77, 'nasty': 2073, 'stuffing': 3093, 'raised': 2504, 'giant': 1390, 'grandma': 1436, 'levels': 1831,
         'pain': 2238, 'surrounded': 3140, 'warning': 3481, 'takeout': 3165, 'cookies': 770, 'york': 3621,
         'remind': 2586, 'dense': 911, 'edges': 1068, 'harder': 1501, 'smile': 2902, 'machine': 1910, 'cash': 535,
         'atm': 246, 'cakes': 500, 'display': 980, 'bakery': 283, 'california': 502, 'charm': 576, 'gold': 1415,
         'enormous': 1105, 'leftover': 1817, 'rave': 2520, 'rough': 2663, 'bummer': 471, 'catching': 539,
         'colleagues': 675, '23': 18, 'lemonade': 1823, 'sugary': 3112, 'romantic': 2656, 'pics': 2327, 'wasted': 3488,
         'wowed': 3592, 'commented': 696, 'famous': 1203, 'ownership': 2228, 'detail': 935, 'kinds': 1759,
         'instantly': 1684, 'breads': 431, 'addicted': 83, 'comfort': 689, 'ny': 2137, 'lastly': 1792, 'straw': 3076,
         'shirt': 2823, 'stole': 3062, 'remained': 2580, 'importantly': 1638, 'become': 328, 'selling': 2780,
         'smiling': 2904, 'force': 1309, 'forced': 1310, 'rules': 2676, 'payment': 2284, 'pastry': 2272,
         'scrumptious': 2745, 'cookie': 769, 'skipped': 2883, 'divine': 985, 'chestnut': 602, 'loyal': 1899,
         'pudding': 2477, 'pastries': 2271, 'discover': 973, 'boot': 401, 'mozzarella': 2052, 'upgrade': 3395,
         'sticky': 3056, 'buns': 474, 'purchased': 2483, 'father': 1215, '28': 23, 'tag': 3161, 'splurge': 2990,
         'decorations': 887, 'advance': 98, 'required': 2604, 'confirm': 733, 'refuse': 2562, 'minimum': 2011,
         'entrance': 1115, 'raspberry': 2513, 'yellow': 3612, 'tart': 3174, 'crusty': 833, 'peanut': 2287, 'crust': 832,
         'tempted': 3194, 'parking': 2253, 'replace': 2594, 'floors': 1289, 'scallops': 2733, 'blueberry': 389,
         'land': 1787, 'pandemic': 2246, 'threw': 3243, 'indulge': 1663, 'sweets': 3148, 'exception': 1151,
         'obsessed': 2140, 'cinnamon': 636, 'quaint': 2491, 'heading': 1513, 'topping': 3285, 'pan': 2243,
         'cutting': 853, 'runny': 2680, 'hint': 1559, 'shape': 2811, 'darn': 863, 'bet': 351, 'budget': 463, '200': 15,
         'fed': 1229, 'espresso': 1126, 'closes': 658, 'sooner': 2941, 'upbeat': 3392, 'shelf': 2819, 'pies': 2333,
         'ride': 2636, 'ave': 262, 'san': 2706, 'art': 227, 'enthusiastic': 1112, 'delivers': 909, 'university': 3381,
         'students': 3090, 'hello': 1533, 'exact': 1145, 'guarantee': 1465, '24': 19, 'attached': 249, 'form': 1317,
         'shops': 2830, 'reputation': 2599, 'cousin': 793, 'judge': 1730, 'watery': 3498, 'purchase': 2482,
         'liquid': 1849, 'boss': 408, 'reliable': 2578, 'exceeded': 1148, 'car': 518, 'concern': 727, 'necessary': 2083,
         'drove': 1034, 'excuse': 1157, 'cheddar': 589, 'disappoints': 970, 'vibrant': 3438, 'layers': 1803,
         'tooth': 3282, 'cancelled': 512, 'buttery': 489, 'ass': 238, 'deserts': 921, 'dough': 1001, 'refund': 2561,
         'whose': 3541, 'primarily': 2440, 'gourmet': 1427, 'ripped': 2643, 'deciding': 882, 'greatly': 1445,
         'freezer': 1330, 'penny': 2291, 'receipt': 2541, 'candy': 514, 'load': 1865, 'stress': 3081,
         'unassuming': 3360, 'memories': 1978, 'loudly': 1887, 'fight': 1246, 'ups': 3398, 'deserves': 924,
         'parked': 2252, 'sparse': 2964, 'drizzle': 1029, 'restrooms': 2618, 'whom': 3540, 'pressed': 2427,
         'feast': 1224, 'match': 1947, 'magic': 1913, 'yogurt': 3620, 'booked': 400, 'freakin': 1327, 'bummed': 470,
         'wound': 3590, 'fridge': 1340, 'america': 160, 'details': 936, '22': 17, 'praise': 2412, 'promise': 2456,
         'celebration': 551, 'donuts': 996, 'assured': 244, 'calls': 506, 'lesson': 1825, 'attended': 252, 'snob': 2915,
         'heated': 1524, 'napkin': 2070, 'soups': 2954, 'smiles': 2903, 'noted': 2126, 'east': 1055,
         'embarrassed': 1086, 'steer': 3047, 'remarkable': 2583, 'impeccable': 1636, 'ladies': 1782, 'delight': 902,
         'speaking': 2966, 'theirs': 3218, 'valentine': 3413, 'living': 1862, 'boys': 425, 'compelled': 709,
         'nature': 2077, 'passing': 2266, 'hi': 1548, 'compliment': 721, 'among': 162, 'goodies': 1419, 'anybody': 177,
         'everybody': 1139, 'combinations': 683, 'tradition': 3302, 'jumbo': 1736, 'turning': 3339, 'sorts': 2947,
         'pleasure': 2369, 'spaghetti': 2961, 'smart': 2898, 'rock': 2649, 'bitter': 370, 'products': 2453,
         'park': 2251, 'current': 843, 'younger': 3623, 'expectation': 1163, 'attempted': 251, 'decadent': 877,
         'max': 1950, 'reminiscent': 2589, 'beaten': 323, 'path': 2273, 'wild': 3546, 'delicate': 898, 'energy': 1097,
         'starts': 3027, 'starters': 3025, 'chops': 627, 'atop': 248, 'splitting': 2989, 'hangout': 1491, 'push': 2486,
         'ravioli': 2523, 'stone': 3064, 'scallop': 2732, 'asian': 230, 'treatment': 3315, 'locations': 1873,
         'warmed': 3477, 'concoction': 731, 'dinning': 958, 'pants': 2247, 'recognize': 2549, 'lobster': 1867,
         'loss': 1882, 'words': 3575, 'language': 1788, 'diners': 954, 'north': 2120, '70': 40, 'remains': 2582,
         'american': 161, 'cheeses': 595, 'condiments': 732, 'setting': 2801, 'magical': 1914, 'elegant': 1081,
         'sun': 3121, 'truffle': 3328, 'leg': 1819, 'prosciutto': 2465, 'blood': 382, 'banana': 289, 'sooooo': 2944,
         'culinary': 838, 'genius': 1383, 'dress': 1017, 'roughly': 2664, 'tries': 3321, 'cooler': 774, 'timing': 3260,
         'avenue': 263, 'pairing': 2241, 'ive': 1713, 'bones': 396, 'cheerful': 590, 'laugh': 1798, 'exquisite': 1179,
         'im': 1631, 'remembered': 2585, 'sounding': 2951, 'mixture': 2028, 'fear': 1223, 'criticism': 824,
         'washington': 3484, 'chilled': 613, 'dancing': 860, 'sushi': 3142, 'social': 2919, 'masks': 1945,
         'flies': 1286, 'reservation': 2605, 'tail': 3162, 'pizzas': 2346, 'interest': 1690, 'consisted': 743,
         'starter': 3024, 'buds': 464, 'dipped': 960, 'drizzled': 1030, 'highlights': 1554, 'sometime': 2933,
         'complex': 720, 'hospitality': 1596, 'steps': 3051, 'individual': 1661, 'sucker': 3108, 'mango': 1930,
         'shout': 2837, 'naturally': 2076, 'crowds': 828, 'sticks': 3055, 'foodie': 1302, 'alot': 147, 'history': 1564,
         'stools': 3066, 'attractive': 257, 'unpleasant': 3385, 'chop': 625, 'crave': 809, 'proud': 2467,
         'versions': 3434, 'sets': 2800, 'generously': 1382, 'freezing': 1331, 'strongly': 3087, 'jersey': 1721,
         'ricotta': 2634, 'spices': 2981, 'caesar': 496, 'foodies': 1303, 'finest': 1259, 'joy': 1729,
         'nevertheless': 2095, 'action': 78, 'exceptionally': 1153, 'vacation': 3412, 'northern': 2122, 'weekly': 3514,
         'holds': 1572, 'steaks': 3041, 'downright': 1005, 'lead': 1806, 'challenge': 562, 'greeting': 1451,
         'answering': 174, 'adventure': 100, 'ideal': 1626, 'stepped': 3050, 'garlicky': 1374, 'penn': 2290,
         'saving': 2725, 'pleasing': 2368, 'officially': 2158, 'intimate': 1694, 'effect': 1071, 'rainy': 2503,
         'downstairs': 1007, 'clams': 641, 'according': 69, 'steamed': 3044, 'minimal': 2010, 'coworkers': 799,
         '3pm': 28, 'grand': 1435, 'parmesan': 2255, 'addictive': 84, 'alcoholic': 132, 'herself': 1544, 'rum': 2677,
         'staying': 3038, 'ease': 1052, 'considered': 741, 'dinners': 957, 'accepted': 58, 'beforehand': 338,
         'strip': 3084, 'devoured': 937, 'featured': 1226, 'dates': 865, 'ending': 1094, 'dropping': 1033,
         'sundays': 3123, 'jerk': 1720, 'bags': 281, 'dude': 1038, 'plans': 2354, 'human': 1610, 'body': 391, 'cc': 547,
         'didnt': 941, 'steal': 3042, 'smelled': 2900, 'attending': 253, 'accommodated': 64, 'tone': 3277,
         'company': 703, 'genuine': 1385, 'spoiled': 2991, 'limit': 1845, 'lunches': 1906, 'therefore': 3223,
         'recommending': 2554, 'continues': 758, 'spicier': 2982, 'caused': 546, 'island': 1704, 'bare': 297,
         'screen': 2743, 'listening': 1854, 'poisoning': 2379, 'inform': 1668, 'fake': 1196, 'semi': 2782,
         'rolled': 2652, 'prime': 2441, 'firm': 1267, 'staffed': 3009, 'entering': 1110, 'rolling': 2653,
         'disgusted': 975, 'exploring': 1177, 'learn': 1809, 'spoon': 2993, 'revisit': 2629, 'sampler': 2703,
         'unprofessional': 3387, '10pm': 3, 'solely': 2926, 'fishy': 1270, 'gooey': 1422, 'automatically': 260,
         'gratuity': 1440, 'fond': 1301, 'gonna': 1418, 'smells': 2901, 'asap': 229, 'supposedly': 3133,
         'cilantro': 635, 'satisfaction': 2711, 'reached': 2527, 'badly': 278, 'steaming': 3045, 'coupon': 789,
         'coat': 665, 'play': 2361, 'ultimately': 3354, 'kindly': 1758, 'midnight': 1997, 'sloppy': 2892,
         'lounge': 1888, 'thinks': 3232, 'blended': 378, 'switched': 3152, 'ruined': 2674, 'incident': 1649,
         'haha': 1475, 'rocks': 2650, 'saved': 2724, 'marinara': 1937, 'hence': 1539, 'vs': 3450, 'rudely': 2672,
         'garnish': 1375, 'skimp': 2879, 'easier': 1053, 'rated': 2515, 'funky': 1362, 'colored': 678, 'arm': 218,
         'sauteed': 2721, 'eight': 1078, 'pinch': 2337, 'lower': 1898, 'accurate': 72, 'washed': 3483, 'manner': 1932,
         'wing': 3554, 'wraps': 3595, 'dr': 1010, 'sleep': 2885, 'halfway': 1478, 'abundance': 54, 'fruity': 1352,
         'promised': 2457, 'reheated': 2572, 'sans': 2709, 'song': 2938, 'sing': 2864, 'encountered': 1090, 'esp': 1124,
         'blew': 379, 'fusion': 1365, 'desire': 927, 'played': 2362, 'jump': 1737, 'lil': 1843, 'record': 2555,
         'lazy': 1805, 'prepare': 2420, 'ruin': 2673, 'squid': 3005, 'present': 2424, 'dealing': 873, 'yell': 3609,
         'mr': 2053, 'bananas': 290, 'bomb': 394, 'charges': 574, 'luke': 1903, 'un': 3356, 'proximity': 2473,
         'central': 555, 'replaced': 2595, 'bowls': 420, 'smothered': 2911, 'hospitable': 1594, 'relaxing': 2577,
         'engaging': 1098, 'explore': 1176, 'streets': 3080, 'unreal': 3388, 'country': 786, 'snow': 2916,
         'shaped': 2812, 'reviewed': 2624, 'separately': 2787, 'earned': 1050, 'grateful': 1439, 'summary': 3119,
         'breath': 436, 'dare': 861, 'peak': 2286, 'comforting': 692, 'frustrating': 1354, 'boston': 409,
         'dripping': 1025, 'fyi': 1368, 'superior': 3127, 'adore': 95, 'accident': 61, 'depends': 914, 'pro': 2444,
         'overrated': 2221, 'exotic': 1161, 'cleaner': 648, 'barbecue': 296, 'text': 3207, 'http': 1606, 'adult': 96,
         'kill': 1753, 'chilly': 614, 'inspired': 1682, 'consists': 747, 'fortunately': 1320, 'carefully': 527,
         'paste': 2270, 'approach': 208, 'remain': 2579, 'humble': 1611, 'torn': 3288, 'entertaining': 1111,
         'metal': 1991, 'claimed': 640, 'require': 2603, 'mmm': 2029, 'mill': 2004, 'bangin': 293, 'hated': 1507,
         'toilet': 3271, 'shaved': 2817, 'thankful': 3213, 'ale': 133, 'triple': 3323, 'negative': 2088,
         'saturdays': 2716, '5pm': 36, 'access': 59, 'complement': 716, 'hassle': 1505, 'www': 3602, 'cuts': 852,
         'appetite': 200, 'cardboard': 522, 'thoughts': 3241, 'bike': 363, '6pm': 39, 'fuck': 1357, 'fucking': 1358,
         'argue': 217, 'soo': 2939, 'fuss': 1366, 'frills': 1348, 'appears': 199, 'broken': 450, 'soul': 2948,
         'stands': 3016, 'wear': 3503, 'skills': 2878, 'piled': 2336, 'stating': 3033, 'chosen': 630, 'randomly': 2508,
         'concert': 729, 'flavoring': 1283, 'crew': 821, 'hangover': 1492, 'grace': 1431, 'ppl': 2409, 'purpose': 2485,
         'insisted': 1681, 'idk': 1627, 'uber': 3351, 'ur': 3403, 'jalapeño': 1716, 'hesitant': 1545, 'native': 2074,
         'premium': 2417, 'friendliest': 1343, 'garbage': 1371, 'grandmother': 1437, 'brief': 441, 'buddy': 462,
         'ingredient': 1673, 'soak': 2917, 'hesitate': 1546, 'tasteful': 3178, 'excessive': 1154, 'prob': 2445,
         'everytime': 1143, 'wash': 3482, 'jack': 1714, 'laughing': 1800, 'uses': 3407, 'yuck': 3625, 'mile': 2001,
         'wasting': 3489, 'pancake': 2244, 'enjoys': 1104, 'practice': 2411, 'cheesesteaks': 597, 'secondly': 2761,
         'asks': 235, 'truck': 3326, 'meets': 1970, 'public': 2476, 'sake': 2691, 'ah': 121, 'shut': 2844,
         'pickup': 2325, 'teas': 3187, 'stores': 3072, 'lacks': 1781, 'string': 3083, 'tastebuds': 3176,
         'definately': 890, 'honor': 1582, 'death': 876, 'neighbor': 2089, 'cart': 532, 'regarding': 2564,
         'powder': 2407, 'speedy': 2975, 'conveniently': 763, 'meatball': 1962, 'cared': 525, 'boo': 398,
         'patience': 2275, 'info': 1667, 'constant': 748, 'pouring': 2406, 'broke': 449, 'explaining': 1174,
         'scared': 2734, 'culture': 839, 'macaroni': 1909, 'arrives': 225, 'code': 671, 'chinatown': 615, 'dive': 983,
         'warn': 3479, 'byo': 493, 'somebody': 2929, 'shy': 2845, 'category': 540, 'invited': 1699, 'zone': 3629,
         'shitty': 2825, 'cleanliness': 650, 'responded': 2611, 'passion': 2267, 'spit': 2987, 'chinese': 616,
         'dumplings': 1041, 'byob': 494, 'doughy': 1002, 'sriracha': 3006, 'sprinkled': 2999, 'similarly': 2860,
         'authenticity': 259, 'helps': 1538, 'mask': 1944, 'convenience': 761, 'seasonings': 2755, 'spiciness': 2983,
         'driver': 1027, 'steam': 3043, 'managers': 1929, 'behavior': 342, 'hub': 1607, 'remove': 2590,
         'hospital': 1595, 'supporting': 3130, 'campus': 510, 'walks': 3464, 'northeast': 2121, 'packaged': 2234,
         'businesses': 485, 'flour': 1290, 'fly': 1293, 'department': 912, 'sale': 2694, 'smoking': 2908,
         'gloves': 1409, 'advise': 105, 'chatted': 580, 'confusion': 737, 'pepperoni': 2294, 'yelled': 3610, 'el': 1080,
         'assorted': 239, 'hoagies': 1569}
reverse_words = {v:k for k,v in words.items()}

  0%|          | 10/2000 [1:53:00<374:50:17, 678.10s/it]


In [163]:
data = pd.read_csv("tf_idf_matrix_2.csv")
data

,name,0,1,2,3,4,5,6,7,8,...,3621,3622,3623,3624,3625,3626,3627,3628,3629,3630
0,Bainbridge Street Barrel House,0.002484,0.030646,0.005444,0.0,0.013573,0.010144,0.005903,0.004414,0.013353,...,0.000000,0.006347,0.000000,0.004531,0.0,0.005088,0.009608,0.003068,0.0,0.000000
1,Farmer's Keep,0.000000,0.075228,0.009481,0.0,0.034382,0.041877,0.006853,0.006834,0.014311,...,0.000000,0.002948,0.000000,0.010522,0.0,0.008861,0.019832,0.000000,0.0,0.010231
2,Kaufman's Delicatessen,0.012030,0.025441,0.000000,0.0,0.011950,0.032749,0.014292,0.014251,0.009948,...,0.080854,0.000000,0.000000,0.021943,0.0,0.000000,0.010340,0.000000,0.0,0.000000
3,The Night Kitchen,0.005717,0.001343,0.000000,0.0,0.000000,0.003459,0.002264,0.000000,0.001576,...,0.002562,0.007791,0.003096,0.005214,0.0,0.003903,0.016380,0.002353,0.0,0.003380
4,Jansen,0.007329,0.012916,0.012046,0.0,0.003640,0.006651,0.000000,0.004341,0.000000,...,0.000000,0.007490,0.000000,0.000000,0.0,0.007505,0.015748,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5047,Istanbul Grill & Cafe,0.000000,0.000000,0.000000,0.0,0.000000,0.045012,0.000000,0.000000,0.041019,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
5048,Bocci's Steakhouse & Comedy Cafe,0.000000,0.000000,0.000000,0.0,0.000000,0.039459,0.000000,0.000000,0.035958,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.053697,0.0,0.000000
5049,Montego Grill,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.045630,0.0,0.000000,0.000000,0.000000,0.0,0.000000
5050,Angelina's Panini Bar,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000


In [164]:
restaurants = data["name"]

In [165]:
matrix = data.drop(["name"], axis=1).to_numpy()

In [166]:
scaler = StandardScaler()
scaled_matrix = scaler.fit_transform(matrix)

In [167]:
pca = PCA(n_components=10)
components = pca.fit_transform(matrix)

In [168]:
components_df = pd.DataFrame(components)
components_df.index = restaurants
components_df

,0,1,2,3,4,5,6,7,8,9
name,,,,,,,,,,
Bainbridge Street Barrel House,-0.237929,0.440945,0.032320,0.111817,0.121645,-0.095972,0.003201,0.110261,0.110737,0.108039
Farmer's Keep,-0.104833,-0.021081,0.004270,-0.081134,-0.050210,0.006705,0.070357,-0.059321,0.105839,0.119862
Kaufman's Delicatessen,-0.084915,-0.042381,0.076369,-0.069765,-0.092400,-0.068219,0.122845,0.001205,0.122539,-0.081684
The Night Kitchen,-0.060405,-0.052921,0.161376,0.007596,-0.133618,0.066631,-0.124952,-0.171109,-0.176238,-0.103596
Jansen,-0.261047,0.276402,-0.161578,-0.093234,-0.243299,0.066774,0.018251,-0.088814,-0.002400,-0.047862
...,...,...,...,...,...,...,...,...,...,...
Istanbul Grill & Cafe,0.013896,-0.052531,-0.020044,-0.025743,0.018260,-0.004494,-0.026680,-0.064646,-0.117193,-0.031710
Bocci's Steakhouse & Comedy Cafe,-0.003427,-0.001881,0.019238,0.036651,0.066297,-0.010771,0.019056,-0.018008,-0.261460,-0.128585
Montego Grill,-0.002555,-0.196437,-0.074725,-0.113660,0.123464,-0.029633,0.064181,-0.088183,-0.225644,0.126130


In [169]:
pd.DataFrame([[reverse_words[int(index)] for index in data[data["name"].isin(components_df.sort_values(by=x, ascending=False).index[:10])].drop(["name"], axis=1).sum().sort_values(ascending=False).iloc[:15].index] for x in tqdm(range(0, 10))])

100%|██████████| 10/10 [00:00<00:00, 27.60it/s]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,pizza,crust,slice,pie,cheese,delivery,slices,sauce,pepperoni,city,pizzas,thin,always,pies,toppings
1,our,pizza,bar,drinks,us,beer,night,she,restaurant,menu,came,table,brunch,server,bartender
2,coffee,bagel,shop,breakfast,iced,sandwiches,sandwich,friendly,cafe,la,staff,cup,love,pastries,always
3,sushi,rolls,roll,tuna,fish,fresh,spicy,byob,salmon,special,restaurant,rice,lunch,our,avocado
4,bar,beer,wings,bartender,beers,dive,night,selection,bartenders,tacos,drinks,drink,pub,music,cheap
5,tacos,mexican,taco,burrito,al,salsa,chips,authentic,delicious,fresh,chorizo,quesadilla,philly,chicken,steak
6,cheesesteak,steak,cheese,fries,philly,sandwich,cheesesteaks,steaks,meat,provolone,onions,sandwiches,chicken,bread,hoagie
7,breakfast,diner,toast,eggs,coffee,our,fries,she,friendly,waitress,pancakes,bacon,staff,brunch,egg
8,sandwich,sandwiches,beer,selection,pork,bread,hoagie,italian,beers,deli,delicious,philly,fresh,friendly,tofu
9,indian,chicken,curry,rice,spicy,lamb,garlic,delicious,delivery,restaurant,sauce,definitely,again,spice,flavor


In [170]:
attributes = pd.read_csv("../business_attributes.csv")
final = pd.merge(left=attributes, right=components_df, left_on="name", right_on="name")

In [171]:
final = final.rename(columns={
    0: "crunchy",
    2: "morning",
    3: "fishy",
    4: "nightlife",
    5: "hearty",
    6: "meaty",
    7: "homey",
    8: "fresh",
    9: "flavorful",
})
final = final.drop([1], axis=1)
final

,Unnamed: 0,name,address,postal_code,stars,categories,useful_review,useful_count,top_10_words,crunchy,morning,fishy,nightlife,hearty,meaty,homey,fresh,flavorful
0,0,St Honore Pastries,935 Race St,19107.0,4.0,Restaurants|Food|Bubble Tea|Coffee & Tea|Bakeries,I really enjoyed my brief visit to St.Honore P...,35,tarts; buns; egg; pastries; chinatown; cake; b...,-0.046408,0.086178,-0.028755,-0.050846,-0.038517,-0.178319,-0.069648,0.005855,-0.127077
1,1,Tuna Bar,205 Race St,19106.0,4.0,Sushi Bars|Restaurants|Japanese,Finally got a chance to check this place out f...,32,sushi; tuna; roll; rolls; bar; fish; sashimi; ...,-0.169896,-0.341037,0.584285,-0.198712,0.067602,0.086788,0.024224,0.032171,0.052671
2,2,BAP,1224 South St,19147.0,4.5,Korean|Restaurants,I've been told I have nice BAPs. I have no ide...,14,bibimbap; korean; beef; rice; stone; bowl; pan...,-0.105530,-0.103943,-0.155096,-0.007175,-0.011044,-0.061221,-0.030621,0.147865,0.120920
3,3,Bar One,767 S 9th St,19147.0,4.0,Cocktail Bars|Bars|Italian|Nightlife|Restaurants,$6.50 for a Kenzinger and the bartender didn't...,7,bar; brunch; beer; ralph; drink; italian; drin...,-0.189365,0.080712,0.163288,0.170747,-0.078686,-0.059158,-0.029006,0.109897,0.061924
4,4,DeSandro on Main,4105 Main St,19127.0,3.0,Pizza|Restaurants|Salad|Soup,Secura? Sacada? I'm not sure. But you know who...,3,pizza; manayunk; drunk; delivery; main; pasta;...,0.530844,-0.091042,-0.022670,-0.032605,-0.030812,-0.002859,0.071702,-0.041031,-0.046296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5038,5047,Stina Pizzeria,1705 Snyder Ave,19145.0,4.5,Pizza|Restaurants|Mediterranean,Stina is without a doubt worthy all the hype!!...,23,pizza; octopus; turkish; mediterranean; caulif...,0.287442,-0.151141,-0.106364,-0.224152,0.106212,-0.043410,-0.127336,0.090170,0.052630
5039,5048,Flip-N-Pizza,1308 W Girard Ave,19123.0,4.0,Restaurants|American (Traditional)|Chicken Win...,I ordered through Uber Eats. The food was prep...,3,pizza; toppings; dough; crust; pick; uncooked;...,0.711622,-0.068431,0.022303,-0.048797,0.108000,-0.110705,-0.051194,0.020871,0.051905
5040,5049,Ali Baba Magic Food,37TH And Walnut,19104.0,4.0,Restaurants|Food Stands,Free taste.\nStill don't know what it was.\nIt...,2,falafel; baba; truck; campus; magic; trucks; h...,0.036707,0.100792,-0.049050,0.076645,0.038409,0.072671,-0.156994,-0.011683,0.067035
5041,5050,Tata Cafe,7201 Germantown Ave,19119.0,4.0,Sandwiches|Restaurants|Italian,Neighborhood gem! \n\nWe appreciate the promp...,3,coffee; pastries; iced; cafe; latte; barista; ...,-0.057980,0.430621,0.066954,-0.257047,0.042047,-0.385558,0.015487,-0.071747,0.094099


In [179]:
final["crunchy"] = (final["crunchy"] - final["crunchy"].min()) / (final["crunchy"] - final["crunchy"].min()).max()
final["morning"] = (final["morning"] - final["morning"].min()) / (final["morning"] - final["morning"].min()).max()
final["fishy"] = (final["fishy"] - final["fishy"].min()) / (final["fishy"] - final["fishy"].min()).max()
final["nightlife"] = (final["nightlife"] - final["nightlife"].min()) / (final["nightlife"] - final["nightlife"].min()).max()
final["hearty"] = (final["hearty"] - final["hearty"].min()) / (final["hearty"] - final["hearty"].min()).max()
final["meaty"] = (final["meaty"] - final["meaty"].min()) / (final["meaty"] - final["meaty"].min()).max()
final["homey"] = (final["homey"] - final["homey"].min()) / (final["homey"] - final["homey"].min()).max()
final["flavorful"] = (final["flavorful"] - final["flavorful"].min()) / (final["flavorful"] - final["flavorful"].min()).max()
final["fresh"] = (final["fresh"] - final["fresh"].min()) / (final["fresh"] - final["fresh"].min()).max()


In [182]:
final = final.drop(["Unnamed: 0"], axis=1)
final

,name,address,postal_code,stars,categories,useful_review,useful_count,top_10_words,crunchy,morning,fishy,nightlife,hearty,meaty,homey,fresh,flavorful
0,St Honore Pastries,935 Race St,19107.0,4.0,Restaurants|Food|Bubble Tea|Coffee & Tea|Bakeries,I really enjoyed my brief visit to St.Honore P...,35,tarts; buns; egg; pastries; chinatown; cake; b...,0.216791,0.566226,0.246221,0.408238,0.245314,0.291937,0.317087,0.490765,0.305616
1,Tuna Bar,205 Race St,19106.0,4.0,Sushi Bars|Restaurants|Japanese,Finally got a chance to check this place out f...,32,sushi; tuna; roll; rolls; bar; fish; sashimi; ...,0.106729,0.145421,0.867981,0.221423,0.350506,0.591871,0.463185,0.527885,0.559671
2,BAP,1224 South St,19147.0,4.5,Korean|Restaurants,I've been told I have nice BAPs. I have no ide...,14,bibimbap; korean; beef; rice; stone; bowl; pan...,0.164097,0.378957,0.118082,0.463413,0.272547,0.424417,0.377827,0.691076,0.656133
3,Bar One,767 S 9th St,19147.0,4.0,Cocktail Bars|Bars|Italian|Nightlife|Restaurants,$6.50 for a Kenzinger and the bartender didn't...,7,bar; brunch; beer; ralph; drink; italian; drin...,0.089376,0.560842,0.440995,0.688203,0.205497,0.426752,0.380341,0.637522,0.572749
4,DeSandro on Main,4105 Main St,19127.0,3.0,Pizza|Restaurants|Salad|Soup,Secura? Sacada? I'm not sure. But you know who...,3,pizza; manayunk; drunk; delivery; main; pasta;...,0.731286,0.391665,0.252392,0.431284,0.252952,0.490446,0.537078,0.424631,0.419791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5038,Stina Pizzeria,1705 Snyder Ave,19145.0,4.5,Pizza|Restaurants|Mediterranean,Stina is without a doubt worthy all the hype!!...,23,pizza; octopus; turkish; mediterranean; caulif...,0.514346,0.332468,0.167507,0.189281,0.388779,0.444569,0.227306,0.609696,0.559613
5039,Flip-N-Pizza,1308 W Girard Ave,19123.0,4.0,Restaurants|American (Traditional)|Chicken Win...,I ordered through Uber Eats. The food was prep...,3,pizza; toppings; dough; crust; pick; uncooked;...,0.892410,0.413937,0.298004,0.410828,0.390551,0.368433,0.345809,0.511946,0.558588
5040,Ali Baba Magic Food,37TH And Walnut,19104.0,4.0,Restaurants|Food Stands,Free taste.\nStill don't know what it was.\nIt...,2,falafel; baba; truck; campus; magic; trucks; h...,0.290870,0.580621,0.225637,0.569313,0.321568,0.575899,0.181148,0.466027,0.579973
5041,Tata Cafe,7201 Germantown Ave,19119.0,4.0,Sandwiches|Restaurants|Italian,Neighborhood gem! \n\nWe appreciate the promp...,3,coffee; pastries; iced; cafe; latte; barista; ...,0.206477,0.905499,0.343290,0.147721,0.325174,0.057472,0.449588,0.381304,0.618224


In [183]:
final.to_csv("business_attributes_vibes.csv")